## RL Text-to-SQL Training on Google Colab

## Setup

In [1]:
# Check GPU
!nvidia-smi

Sun Feb 15 07:14:49 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
%cd /kaggle/working
!rm -rf rl-text2sql
!git clone https://github.com/GameChaser782/rl-text2sql.git
%cd rl-text2sql

/kaggle/working
Cloning into 'rl-text2sql'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 85 (delta 42), reused 64 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 52.54 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/kaggle/working/rl-text2sql


In [3]:
!pip install -r requirements.txt

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-kibr6_9c/unsloth_9b940cb0496a4ed49af6783b659f6d55
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-kibr6_9c/unsloth_9b940cb0496a4ed49af6783b659f6d55
  Resolved https://github.com/unslothai/unsloth.git to commit 86fde7603aa0fa6f2a3a92b3f29088a8a910e0e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 99.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 107.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.6 MB/s eta 0:00:00
  Created 

## Download Spider Dataset

In [17]:
import json
import os
from datasets import load_dataset

os.makedirs('data/spider', exist_ok=True)
os.makedirs('results', exist_ok=True)

dataset = load_dataset("spider")

train_data = [{'question': item['question'], 'query': item['query'], 'db_id': item['db_id']} 
              for item in dataset['train'].select(range(30))]

dev_data = [{'question': item['question'], 'query': item['query'], 'db_id': item['db_id']} 
            for item in dataset['validation'].select(range(10))]

with open('data/spider/train_30.json', 'w') as f:
    json.dump(train_data, f)
with open('data/spider/dev_10.json', 'w') as f:
    json.dump(dev_data, f)

print(f"✅ {len(train_data)} train, {len(dev_data)} dev examples")

✅ 30 train, 10 dev examples


In [5]:
!python evaluate.py \
    --model_path Qwen/Qwen2.5-Coder-3B-Instruct \
    --base_model Qwen/Qwen2.5-Coder-3B-Instruct \
    --test_data data/spider/dev_10.json \
    --db_root data/spider/database \
    --num_gpus 1 \
    --output_file results/baseline.json

2026-02-15 07:16:21.732619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771139782.104822     199 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771139782.205237     199 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771139783.025087     199 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771139783.025131     199 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771139783.025138     199 computation_placer.cc:177] computation placer alr

## Run Training

In [10]:
!pip install unsloth[kaggle]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 9.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 85.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      Successfully uninstalled torchao-0.10.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling

In [11]:
!pip install "pyarrow<20.0.0" "pydantic<2.12" "rich<14" "fsspec==2025.3.0"
!pip install "google-cloud-bigquery-storage<3.0.0,>=2.30.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 47.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.5
    Uninstalling pydantic_core-2.41.5:
      Successfully uninstalled pydantic_core-2.41.5
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 22.0.0
    Uninstalling pyarrow-22.0.0:
      Successfully uninstalled pyarrow-22.0.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0
  Attemp

In [20]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

In [21]:
print("="*70)
print("🎯 RL TRAINING")
print("="*70)

!python train_rl.py \
    --config config_grpo_spider.yaml \
    --train_data data/spider/train_30.json \
    --db_root data/spider/database \
    --num_gpus 1 \
    --use_unsloth

print("\n✅ Training complete")

🎯 RL TRAINING
/kaggle/working/rl-text2sql/src/models/model_loader.py:3: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-15 07:34:34.572286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771140874.594092     804 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771140874.602750     804 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alr

## Evaluation

In [22]:
!python evaluate.py \
    --model_path outputs/rl-text2sql \
    --base_model Qwen/Qwen2.5-Coder-3B-Instruct \
    --test_data data/spider/dev_10.json \
    --db_root data/spider/database \
    --num_gpus 1 \
    --output_file results/rl_model.json

2026-02-15 08:19:34.192080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771143574.213264    1032 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771143574.219840    1032 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771143574.238816    1032 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771143574.238853    1032 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771143574.238858    1032 computation_placer.cc:177] computation placer alr

In [23]:
import json
import pandas as pd

with open('results/baseline.json') as f:
    baseline = json.load(f)
with open('results/rl_model.json') as f:
    rl = json.load(f)

comparison = pd.DataFrame({
    'Metric': ['Execution Accuracy', 'Exact Match', 'Precision', 'Recall', 'F1'],
    'Baseline': [f"{baseline['metrics']['execution_accuracy']:.1%}", 
                 f"{baseline['metrics']['exact_match']:.1%}",
                 f"{baseline['metrics']['precision']:.3f}",
                 f"{baseline['metrics']['recall']:.3f}",
                 f"{baseline['metrics']['f1_score']:.3f}"],
    'After RL': [f"{rl['metrics']['execution_accuracy']:.1%}",
                 f"{rl['metrics']['exact_match']:.1%}",
                 f"{rl['metrics']['precision']:.3f}",
                 f"{rl['metrics']['recall']:.3f}",
                 f"{rl['metrics']['f1_score']:.3f}"]
})

print(comparison.to_string(index=False))

# Sample predictions
for i, pred in enumerate(rl['predictions'][:3], 1):
    print(f"\nExample {i}:")
    print(f"Q: {pred['question']}")
    print(f"Pred: {pred['pred_sql']}")
    print(f"Gold: {pred['gold_sql']}")
    print(f"✅" if pred['execution_correct'] else "❌")

FileNotFoundError: [Errno 2] No such file or directory: 'results/baseline.json'

## Save to Drive (Optional)

In [24]:
!cp -r outputs results /kaggle/working/
print("✅ Saved to /kaggle/working/")

✅ Saved to /kaggle/working/
